In [3]:
import json
import sys
sys.path.insert(1, "../")
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from parse import parser

In [4]:
parse = parser("../tsv/EVDa_SimCaTip_Ale0003.tsv")
parse.parse()
dataJSON = parse.initData()

In [5]:
data = pd.json_normalize(json.loads(dataJSON)).astype({"FRAME": "int",
                                                     "TIME": "float",
                                                     "X": "float",
                                                     "Y": "float",
                                                     "Z": "float"})
data.head()

,FRAME,TIME,PHANTOM,MARKER_NR,TRAJECTORY_TYPE,X,Y,Z
0,1,0.0,skull,phantom1,Measured,338.365,353.008,183.094
1,1,0.0,skull,phantom2,Measured,302.125,413.543,148.604
2,1,0.0,skull,phantom3,Measured,351.689,347.606,90.599
3,1,0.0,skull,phantom4,Measured,301.656,320.014,154.561
4,1,0.0,catheter,tip1,Measured,303.252,84.580,-20.389


In [6]:
data.sort_values(by=["MARKER_NR", "FRAME"], inplace=True)

In [7]:
data["flag"] = (data.loc[:, ["MARKER_NR"]].shift(-1) == data.loc[:, ["MARKER_NR"]].shift(1))
data[["VX", "VY", "VZ"]] = ((data.loc[:, ["X", "Y", "Z"]].shift(-1) - data.loc[:, ["X", "Y", "Z"]].shift(1)) / 0.01).where(data["flag"], np.nan)
data.drop("flag", axis=1, inplace=True)
data.head()

,FRAME,TIME,PHANTOM,MARKER_NR,TRAJECTORY_TYPE,X,Y,Z,VX,VY,VZ
0,1,0.000,skull,phantom1,Measured,338.365,353.008,183.094,NaN,NaN,NaN
8,2,0.005,skull,phantom1,Measured,338.354,353.005,183.090,-1.3,-0.8,-0.7
16,3,0.010,skull,phantom1,Measured,338.352,353.000,183.087,23.8,41.3,-49.0
24,4,0.015,skull,phantom1,Measured,338.592,353.418,182.600,18.9,45.4,-46.7
32,5,0.020,skull,phantom1,Measured,338.541,353.454,182.620,-5.9,10.1,0.2


In [8]:
data["VELOCITY"] = np.sqrt(data["VX"]**2 + data["VY"]**2 + data["VZ"]**2).rolling(window=3, center=True).mean()
data.head()

,FRAME,TIME,PHANTOM,MARKER_NR,TRAJECTORY_TYPE,X,Y,Z,VX,VY,VZ,VELOCITY
0,1,0.000,skull,phantom1,Measured,338.365,353.008,183.094,NaN,NaN,NaN,NaN
8,2,0.005,skull,phantom1,Measured,338.354,353.005,183.090,-1.3,-0.8,-0.7,NaN
16,3,0.010,skull,phantom1,Measured,338.352,353.000,183.087,23.8,41.3,-49.0,45.952475
24,4,0.015,skull,phantom1,Measured,338.592,353.418,182.600,18.9,45.4,-46.7,49.292286
32,5,0.020,skull,phantom1,Measured,338.541,353.454,182.620,-5.9,10.1,0.2,29.503668


In [9]:
px.line(data, x="TIME", y="VELOCITY", color="MARKER_NR", title="Velocity of each marker over time").show()